# WEEK 6
## Encounter 06 - Visualizing Geospatial Data with Plotly
## Project Challenge - Create an interactive climate map with plotly

### Task Description
Choose one (or more) of these options and plot using plotly:  

* Create animated choropleth map of the temperature of countries over time.  

**Tips:** Referring to the derived table yearly_mean_temperature made in the advanced SQL milestone import the yearly average temp per country, alpha codes and the year from database into a pandas dataframe. A complex join will be neccessary. Use the choropleth plot function from plotly with its animation_frame parameter in order to animate the plot.  

* Create scattermap of all the weather stations in the data.  

**Tips:** Using an inner join import a dataframe into pandas with the station name, alpha country code, and coordinates. Use the scatter_geo function from plotlyto visualize the locations of the each weather station. scatter_geo Documentation.  


In [48]:
import pandas as pd
import os

from dotenv import load_dotenv

from sqlalchemy import create_engine
from sqlalchemy import text  # to be able to pass string

import plotly.express as px

In [49]:
# loading connection values from .env file
load_dotenv()

# define variables for the login
username = os.getenv('USER')
password = os.getenv('PASS')
host = os.getenv('HOST')
port = os.getenv('PORT')

# connection string to the database
url = f'postgresql://{username}:{password}@{host}:{port}/climate'
        
# create a connection to climate database in PostgreSQL
engine = create_engine(url, echo=False)

In [50]:
# reading the data from database and store it as a dataframe

with engine.begin() as conn:
    result = conn.execute(text("SELECT * FROM yearly_mean_temperature_upd;"))
    # reading all rows from the result object and returning a list of tuples
    data = result.all()

In [51]:
# saving loaded data as DataFrame

yearly_temps = pd.DataFrame(data, columns=['country_name', 'country_iso3', 'station_name', 'year', 'yearly_temp_in_grad'])

# cleaning DataFrame values:
#    - getting read of leading and trailing spaces for dtype='object'
#    - converting temperatures (yearly_temp_in_grad) from 'object' to 'float32'

yearly_temps['country_name'] = yearly_temps['country_name'].str.strip()
yearly_temps['country_iso3'] = yearly_temps['country_iso3'].str.strip() 
yearly_temps['station_name'] = yearly_temps['station_name'].str.strip()
yearly_temps['yearly_temp_in_grad'] = pd.to_numeric(yearly_temps['yearly_temp_in_grad'], downcast="float")
yearly_temps['year'] = yearly_temps['year'].astype(int)

yearly_temps

,country_name,country_iso3,station_name,year,yearly_temp_in_grad
0,Sweden,SWE,STOCKHOLM,1756,4.9
1,Sweden,SWE,STOCKHOLM_A,1756,4.9
2,Sweden,SWE,VARTAN,1756,4.9
3,Sweden,SWE,VARTAN,1757,5.9
4,Sweden,SWE,STOCKHOLM_A,1757,5.9
...,...,...,...,...,...
321966,United Kingdom,GBR,MANSTON,2022,9.9
321967,United Kingdom,GBR,HASTINGS,2022,10.2
321968,United Kingdom,GBR,WOBURN,2022,9.4
321969,United Kingdom,GBR,WISLEY,2022,10.1


### Case 1: Create animated choropleth map of the temperature of countries over time

In [52]:
# grouping the data to get avg temperature for contry per year

yearly_temps_per_country = yearly_temps.groupby(['year', 'country_name', 'country_iso3'])[['yearly_temp_in_grad']].mean()
yearly_temps_per_country.reset_index(inplace=True)
yearly_temps_per_country

,year,country_name,country_iso3,yearly_temp_in_grad
0,1756,Sweden,SWE,4.900000
1,1757,Sweden,SWE,5.900000
2,1758,Sweden,SWE,4.800000
3,1759,Sweden,SWE,6.300001
4,1760,Sweden,SWE,5.300000
...,...,...,...,...
6019,2022,Svalbard and Jan Mayen,SJM,-6.785714
6020,2022,Sweden,SWE,3.604296
6021,2022,Switzerland,CHE,7.955555
6022,2022,Ukraine,UKR,6.866667


In [53]:
# Creating animated choropleth map of the temperature of countries over time

fig = px.choropleth(yearly_temps_per_country, locations='country_iso3', 
                    #projection='orthographic',
                    projection='natural earth',
                    scope='world',
                    #scope='europe',
                    #color='country_name',
                    color='yearly_temp_in_grad',
                    color_continuous_scale=px.colors.sequential.Bluered,
                    locationmode='ISO-3', # published by the International Organization for Standardization (ISO)
                    animation_frame='year',
                    #center={'lat': 51.1657, 'lon': 10.4515}, # geographic center of Germany
                    #center={'lat': 53.5289, 'lon': 28.0447}, # geographic center of Belarus
                    title='Animated choropleth map of the temperature of countries over time'
                   ) 
#fig.write_html('countries_avg_tmp_over_years.html')
fig.show()

In [60]:
yearly_temps_per_country

,year,country_name,country_iso3,yearly_temp_in_grad
0,1756,Sweden,SWE,4.900000
1,1757,Sweden,SWE,5.900000
2,1758,Sweden,SWE,4.800000
3,1759,Sweden,SWE,6.300001
4,1760,Sweden,SWE,5.300000
...,...,...,...,...
6019,2022,Svalbard and Jan Mayen,SJM,-6.785714
6020,2022,Sweden,SWE,3.604296
6021,2022,Switzerland,CHE,7.955555
6022,2022,Ukraine,UKR,6.866667


In [64]:
# list of colors - qualitative
fig = px.colors.qualitative.swatches()
fig.show()

In [63]:
# list of colors - sequential
fig = px.colors.sequential.swatches()
fig.show()

In [54]:
# Creating animated choropleth map of the temperature of countries over time

fig = px.choropleth(yearly_temps_per_country, locations='country_iso3', 
                    #projection='orthographic',
                    projection='natural earth',
                    #scope='world',
                    scope='europe',
                    #color='country_name',
                    color='yearly_temp_in_grad',
                    color_continuous_scale=px.colors.sequential.Bluered,
                    locationmode='ISO-3', # published by the International Organization for Standardization (ISO)
                    animation_frame='year',
                    #center={'lat': 51.1657, 'lon': 10.4515}, # geographic center of Germany
                    center={'lat': 53.5289, 'lon': 28.0447}, # geographic center of Belarus
                    title='Animated choropleth map of the temperature of countries over time'
                   ) 
#fig.write_html('countries_avg_tmp_over_years_EU.html')
fig.show()

## Case 2: Create scattermap of all the weather stations in the data.

In [55]:
# read stations_lat_lon data
with engine.begin() as conn:
    result2 = conn.execute(text("SELECT * FROM stations_lat_lon;"))
    # reading all rows from the result object and returning a list of tuples
    data2 = result2.all()

In [56]:
# saving loaded data as DataFrame
stations = pd.DataFrame(data2, columns=['station_name', 'country_name', 'country_iso3', 'lat_formated', 'lon_formated', 'coordinates'])
stations

,station_name,country_name,country_iso3,lat_formated,lon_formated,coordinates
0,SHKODRA,Albania,ALB,42.10000000000000000000,19.53305555555555555556,"(42.1,19.533055555555556)"
1,TIRANA,Albania,ALB,41.33305555555555555556,19.78305555555555555556,"(41.33305555555555,19.783055555555556)"
2,ALGER-DAR EL BEIDA,Algeria,DZA,36.71666666666666666667,3.25000000000000000000,"(36.71666666666667,3.25)"
3,EL-GOLEA,Algeria,DZA,30.56666666666666666667,2.86666666666666666667,"(30.566666666666666,2.8666666666666667)"
4,IN-AMENAS,Algeria,DZA,28.05000000000000000000,9.63305555555555555556,"(28.05,9.633055555555556)"
...,...,...,...,...,...,...
6450,FERGANA,Uzbekistan,UZB,40.36666666666666666667,71.75000000000000000000,"(40.36666666666667,71.75)"
6451,SAMARKAND,Uzbekistan,UZB,39.56666666666666666667,66.95000000000000000000,"(39.56666666666667,66.95)"
6452,TAMDY,Uzbekistan,UZB,41.73305555555555555556,64.61666666666666666667,"(41.73305555555555,64.61666666666666)"
6453,TASHKENT,Uzbekistan,UZB,41.27000000000000000000,69.26944444444444444445,"(41.27,69.26944444444445)"


In [59]:
# plotting stations on the map using latitude and longitude
fig = px.scatter_mapbox(stations, 
                        lat="lat_formated", lon="lon_formated",
                        #locationmode='ISO-3',
                        color="country_name",
                        #size="yearly_temp_in_grad",
                        #animation_frame='year',  # no animation
                        zoom=2, 
                        center={'lat': 53.5289, 'lon': 28.0447}, # Belarus
                        mapbox_style='carto-positron'
                       )
    
fig.write_html('weather_stations_static.html')
fig.show()